In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import *

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

#### Load data...

In [ ]:
train = pd.read_csv("/kaggle/input/tabular-playground-series-feb-2021/train.csv")
test = pd.read_csv("/kaggle/input/tabular-playground-series-feb-2021/test.csv")
sample_submission = pd.read_csv('../input/tabular-playground-series-feb-2021/sample_submission.csv')

### Look at data...

In [ ]:
train.tail()

In [ ]:
train.shape

In [ ]:
test.tail()

In [ ]:
test.shape

In [ ]:
sample_submission['target'].value_counts()

# Do your stuff...

### Deleverables
- EDA
    - What's going on?
    - Show me the data...
    - Bonus: Other / Etc...
- Model
    - [RandomForestRegressor()](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html)
    - Bonus: [KNeighborsRegressor()](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsRegressor.html)
    - Bonus: Other / Etc...
- Evaluation
    - mean_squared_error()
    - Bonus: [roc_auc_score()](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_auc_score.html)
    - Bonus: Other / Etc...
- RAPIDS Bonus
    - Apply RAPIDS ([Starter Notebook](https://www.kaggle.com/tunguz/tps-feb-2021-rapids-starter))
    - Replace pandas with cuDF & sklearn with cuDF & cuML

In [ ]:
# good luck, have fun!

**EDA**

In [ ]:
train.describe(exclude=[np.number])

In [ ]:
train.describe

Correleation Map

In [ ]:
corr = train.corr()
corr.style.background_gradient(cmap='coolwarm')

Pairplot for Train Data

In [ ]:
import seaborn as sns

sns.pairplot(train.sample(100));

Visualizition Outliers in Data

In [ ]:
fig=plt.figure(figsize=(25,15))
train_outliers = train.drop('id', axis=1)
plt.title('Outliers in boxplot chart ')
sns.boxplot(data=train_outliers);

**Model**

In [ ]:
cat_col = [col for col in train.columns if train[col].dtype=='O']
cat_col

In [ ]:
# Seperate train_clean into target and features 
y = train['target']
X_train = train.drop('target',axis = 'columns')

# Save the index for X_train_clean 
X_train_clean_index = X_train.index.to_list()

# Row bind train.csv features with test.csv features 
# this makes it easier to apply label encoding onto the entire dataset 
X_total = X_train.append(test ,ignore_index = True)
# save the index for test.csv 
X_test_clean_index = np.setdiff1d(X_total.index.to_list() ,X_train_clean_index) 

X_total

In [ ]:
cat_col = [col for col in train.columns if train[col].dtype=='O']
cat_col

In [ ]:
categorical = train[cat_col]

In [ ]:
categorical_dum = pd.get_dummies(categorical)
categorical_dum

In [ ]:
df_new = pd.concat(
    [train,
     categorical_dum],
      axis=1)
df_new

In [ ]:
cat_col = [col for col in df_new.columns if df_new[col].dtype=='O']
df_new = df_new.drop(cat_col, axis = 1)
#train.drop(cat_col, axis=1)

In [ ]:
from sklearn.model_selection import train_test_split

# Create test and train set 80-20
#%%  train-test stratified split using a 80-20 split
X_train, X_test, y_train, y_test = train_test_split(df_new, y, test_size=0.3, random_state=0)

**Random Forest Regressor**

In [ ]:
from sklearn.ensemble import RandomForestRegressor

RandomForestRegressor()

rfr = RandomForestRegressor()

rfr.fit(X_train, y_train)

In [ ]:
pred_rfr = rfr.predict(X_test)
pred_rfr

In [ ]:
acc_rfr = r2_score(pd.DataFrame(y_test), pred_rfr)
acc_rfr

**K-Neighbors Regressor**

In [ ]:
from sklearn.neighbors import KNeighborsRegressor

In [ ]:
# instantiate the model and set the number of neighbors to consider to 3
knn = KNeighborsRegressor(n_neighbors=5)
# fit the model using the training data and training targets
knn.fit(X_train, y_train)

In [ ]:
pred_knn = knn.predict(X_test)
pred_knn

In [ ]:
acc_knn = r2_score(y_test, pred_knn)
acc_knn

**Evaluation**

**Mean Squared Error**
* Mean sequared error RFR

In [ ]:
from sklearn.metrics import mean_squared_error

mean_squared_error(y_test, pred_rfr)

* Mean sequared error KNN

In [ ]:
mean_squared_error(y_test, pred_knn)

**Mean Absolute Error**
* Mean absolute error RFR

In [ ]:
from sklearn.metrics import mean_absolute_error

mean_absolute_error(y_test, pred_rfr)

* Mean absolute error KNN

In [ ]:
mean_absolute_error(y_test, pred_knn)